In [131]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define THREADS (1 << 10)
#define BLOCKS (1 << 12)
#define NUM_VALS THREADS * BLOCKS

void printElapsed(clock_t start, clock_t stop) {
    double elapsed = ((double) (stop - start)) / CLOCKS_PER_SEC;
    printf("Elapsed time: %.5fs\n", elapsed);
}

void valuesFill(int *arr, int n) {
    srand(time(NULL));
    for (int i = 0; i < n; ++i) arr[i] = rand();
}

__global__ void bitonicSortGpuStep(int *deviceValues, int j, int k) {
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x, ixj = i ^ j;

    if (ixj > i) {
        if ((i&k) == 0) {
            if (deviceValues[i] > deviceValues[ixj]) {
                int temp = deviceValues[i];
                deviceValues[i] = deviceValues[ixj];
                deviceValues[ixj] = temp;
            }
        }
        else {
            if (deviceValues[i] < deviceValues[ixj]) {
                int temp = deviceValues[i];
                deviceValues[i] = deviceValues[ixj];
                deviceValues[ixj] = temp;
            }
        }
    }
}

void bitonicSortGpu(int *hostValues)
{
    int *deviceValues;
    size_t size = NUM_VALS * sizeof(int);

    cudaError_t cudaError = cudaMalloc((void**) &deviceValues, size);
    if (cudaError != cudaSuccess) {
        fprintf(stderr, "Cannot allocate Gpu memory for deviceValues: %s\n",
                cudaGetErrorString(cudaError));
        exit(1);
    }

    cudaError = cudaMemcpy(deviceValues, hostValues, size, cudaMemcpyHostToDevice);
    if (cudaError != cudaSuccess) {
        fprintf(stderr, "Cannot copy data from hostValues to deviceValues: %s\n",
                cudaGetErrorString(cudaError));
        exit(1);
    }

    dim3 blocks(BLOCKS, 1);
    dim3 threads(THREADS, 1);

    for (int k = 2; k <= NUM_VALS; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortGpuStep<<<blocks, threads>>>(deviceValues, j, k);

            cudaError = cudaGetLastError();
            if (cudaError != cudaSuccess) {
                fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
                        cudaGetErrorString(cudaError));
                exit(1);
            }
        }
    }

    cudaError = cudaMemcpy(hostValues, deviceValues, size, cudaMemcpyDeviceToHost);
    if (cudaError != cudaSuccess) {
        fprintf(stderr, "Cannot copy data from deviceValues to hostValues: %s\n",
                cudaGetErrorString(cudaError));
        exit(1);
    }

    cudaError = cudaFree(deviceValues);
    if (cudaError != cudaSuccess) {
        fprintf(stderr, "Cannot free Gpu memory for deviceValues: %s\n",
                cudaGetErrorString(cudaError));
        exit(1);
    }
}

void compare(int* a, int* b) {
    int ta = *a, tb = *b;
    if (ta > tb) *a = tb, *b = ta;
}

void bitonicSortCpuStep(int* values, int n) {
    int K = log2(n), d = 1 << K; --K;
    for (int i = 0; i < d >> 1; ++i)
        compare(&values[i], &values[d - i - 1]);
    for (int k = K; k > 0; k--) {
        d = 1 << k;
        for (int j = 0; j < n; j += d)
            for (int i = 0; i < d >> 1; ++i)
                compare(&values[j + i], &values[j + (d >> 1) + i]);
    }
}

void bitonicSortCpu(int* values) {
    int *temp = (int*)malloc(NUM_VALS * sizeof(int));
    memcpy(temp, values, NUM_VALS * sizeof(int));
    int K = log2(NUM_VALS);
    for (int k = 1, d = 2; k <= K; ++k, d <<= 1)
        for (int i = 0; i < NUM_VALS; i += d)
            bitonicSortCpuStep((int*)&temp[i], d);
    memcpy(values, temp, NUM_VALS * sizeof(int));
    free(temp);
    return;
}

int main(void)
{
    clock_t start, stop;

    int *hostValues = (int*)malloc(NUM_VALS * sizeof(int)),
        *hostValuesForCpu = (int*)malloc(NUM_VALS * sizeof(int));

    valuesFill(hostValues, NUM_VALS);
    memcpy(hostValuesForCpu, hostValues, NUM_VALS * sizeof(int));

    start = clock();
    bitonicSortGpu(hostValues);
    stop = clock();

    printElapsed(start, stop);

    bool hostValuesIsSorted = true;

    for (int i = 1; i < NUM_VALS; ++i)
        if (hostValues[i] < hostValues[i - 1]) {
            hostValuesIsSorted = false;
            break;
        }

    if (hostValuesIsSorted)
        printf("Gpu sorting correctly\n");
    else
        printf("Gpu sorting incorrectly\n");

    start = clock();
    bitonicSortCpu(hostValuesForCpu);
    stop = clock();

    printElapsed(start, stop);

    bool hostValuesForCpuIsSorted = true;

    for (int i = 1; i < NUM_VALS; ++i)
        if (hostValuesForCpu[i] < hostValuesForCpu[i - 1]) {
            hostValuesForCpuIsSorted = false;
            break;
        }

    if (hostValuesForCpuIsSorted)
        printf("Cpu sorting correctly\n");
    else
        printf("Cpu sorting incorrectly\n");

    int arraysEquality = -1;

    for (int i = 0; i < NUM_VALS; ++i)
        if (hostValues[i] != hostValuesForCpu[i]) {
            arraysEquality = i;
            break;
        }

    if (arraysEquality == -1)
        printf("Gpu values are equal to cpu values\n");
    else {
        printf("Gpu values are not equal to Cpu values at: %d\n", arraysEquality);

        for (int i = 0; i <= arraysEquality; ++i) printf("%d ", hostValues[i]); printf("\n");
        for (int i = 0; i <= arraysEquality; ++i) printf("%d ", hostValuesForCpu[i]); printf("\n");
    }

    free(hostValuesForCpu);
    free(hostValues);

    return 0;
}

Elapsed time: 0.16255s
Gpu sorting correctly
Elapsed time: 4.03825s
Cpu sorting correctly
Gpu values are equal to cpu values

